In [ ]:
import tensorflow as tf
import urllib
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model

# Crea un callback para detener el entrenamiento cuando llegue al 85%
# Your code here

def get_data():
    _TRAIN_URL = "https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip"
    _TEST_URL = "https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip"
    urllib.request.urlretrieve(_TRAIN_URL, 'horse-or-human.zip')
    local_zip = 'horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('data/horse-or-human/')
    zip_ref.close()
    urllib.request.urlretrieve(_TEST_URL, 'testdata.zip')
    local_zip = 'testdata.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('data/testdata/')
    zip_ref.close()

def solution_model():
    train_dir = 'data/horse-or-human'
    validation_dir = 'data/testdata'

    print('total training horses images :', len(os.listdir(os.path.join(train_dir, 'horses'))))
    print('total training humans images :', len(os.listdir(os.path.join(train_dir, 'humans'))))
    print('total validation horses images :', len(os.listdir(os.path.join(validation_dir, 'horses'))))
    print('total validation humans images :', len(os.listdir(os.path.join(validation_dir, 'humans'))))

    # Carga los datos mediante ImageDataGenerator
    # Your code here

    train_datagen = ImageDataGenerator(
                                  #rotation_range = 180,
                                  rescale=1./255,
                                  #shear_range=0.2,
                                  #zoom_range=0.2,        
                                  #fill_mode='nearest',
                                  #horizontal_flip=True,
                                  #vertical_flip=True,
                                  )
    
    test_datagen = ImageDataGenerator(rescale=1./255)

    batch_size = 10

    train_generator = train_datagen.flow_from_directory('/content/data/horse-or-human/',
                                                    target_size = (300, 300),
                                                    batch_size = batch_size,                                                    
                                                    class_mode = 'binary'                                                 
                                                    )

    validation_generator = test_datagen.flow_from_directory('/content/data/testdata',
                                                target_size = (300, 300),
                                                batch_size = batch_size,                                                
                                                class_mode = 'binary'                                                  
                                                )

    model = tf.keras.models.Sequential([
        # Note the input shape specified on your first layer must be (300,300,3)
        # Your Code here
        tf.keras.layers.Conv2D(filters = 32,kernel_size = (3, 3), # Convolution layer
                      input_shape = (300, 300, 3), activation = "relu"),

        tf.keras.layers.MaxPooling2D(pool_size = (2,2)),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(units = 256, activation = "relu"),

        tf.keras.layers.Dense(units = 1, activation="sigmoid")
        
    ])

    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(lr=1e-4),
                  metrics=['accuracy'])
    
    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
          '''
          Halts the training after reaching 85 percent accuracy

          Args:
            epoch (integer) - index of epoch (required but unused in the function definition below)
            logs (dict) - metric results from the training epoch
          '''

          # Check accuracy
          if(logs.get('accuracy') >= 0.85):

            # Stop if threshold is met
            print("\nAccuracy is greater than 0.85, so cancelling training!")
            self.model.stop_training = True

    callbacks = myCallback()

    model.fit(train_generator, 
              epochs=10,
              steps_per_epoch=20, # 6000 img / 20 = 300 img == batch_size
              validation_data=validation_generator,
              verbose=1,
              validation_steps=10,
              callbacks=[callbacks])

    return model

In [ ]:
if __name__ == '__main__':    
    get_data()
    model = solution_model()

total training horses images : 500
total training humans images : 527
total validation horses images : 128
total validation humans images : 128
Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


Epoch 1/10
20/20 [==============================] - 73s 4s/step - loss: 77.0269 - accuracy: 0.5200 - val_loss: 28.2594 - val_accuracy: 0.4400
Epoch 2/10
20/20 [==============================] - 70s 4s/step - loss: 8.2082 - accuracy: 0.6250 - val_loss: 27.6932 - val_accuracy: 0.4800
Epoch 3/10
20/20 [==============================] - 71s 4s/step - loss: 2.6919 - accuracy: 0.7050 - val_loss: 5.0850 - val_accuracy: 0.5300
Epoch 4/10
20/20 [==============================] - ETA: 0s - loss: 0.3673 - accuracy: 0.8985
Accuracy is greater than 0.85, so cancelling training!
20/20 [==============================] - 73s 4s/step - loss: 0.3673 - accuracy: 0.8985 - val_loss: 0.9453 - val_accuracy: 0.7300
